In [0]:
#import libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [0]:
!unzip ml-1m.zip

In [0]:
!unzip ml-100k.zip

In [0]:
#Import dataset
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [0]:
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [0]:
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [0]:
#Prepare training set and test set
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [8]:
#Get number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))
print(nb_users)
print(nb_movies)

943
1682


In [0]:
#Convert data into an array with users in lines and movies in columns
def convert(data):
  #list of list is created and not 2d array
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:,1][data[:,0] == id_users]
    id_ratings = data[:,2][data[:,0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [0]:
#Convert data into Torch Tensors (multidimensional matrix)
training_set = torch.FloatTensor(training_set) 
test_set = torch.FloatTensor(test_set)

In [0]:
#Convert ratings into binary ratings: 1 (Liked) or 0 (Not Liked)
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0 
training_set[training_set >= 3] = 1

test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [0]:
#Create architecture of Neural Network 
class RBM():
  #Init function (default compulsory function)
  def __init__(self, nv, nh):
    self.W = torch.randn(nh, nv) #Weights
    self.a = torch.randn(1, nh) #bias for hidden nodes
    self.b = torch.randn(1, nv) #bias for visible nodes
  def sample_h(self, x):
    #x = vector of visible neurons
    wx = torch.mm(x, self.W.t()) 
    activation = wx + self.a.expand_as(wx) 
    p_h_given_v = torch.sigmoid(activation)
    return p_h_given_v, torch.bernoulli(p_h_given_v)
  def sample_v(self, y):
    #y corresponds to hidden nodes
    wy = torch.mm(y, self.W) 
    activation = wy + self.b.expand_as(wy) 
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h, torch.bernoulli(p_v_given_h)
  def train(self, v0, vk, ph0, phk):
    self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
    self.b += torch.sum((v0 - vk), 0)
    self.a += torch.sum((ph0 - phk), 0)

In [0]:
nv = len(training_set[0])
nh = 100 
batch_size = 100 
rbm = RBM(nv, nh)

In [14]:
#Train RBM
nb_epoch = 10 
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(0, nb_users - batch_size, batch_size):
    vk = training_set[id_user:id_user+batch_size]
    v0 = training_set[id_user:id_user+batch_size]
    ph0,_ = rbm.sample_h(v0)
    for k in range(10):
      _,hk = rbm.sample_h(vk)
      _,vk = rbm.sample_v(hk)
      #freeze visible nodes with -1 ratings
      vk[v0<0] = v0[v0<0]
    phk,_ = rbm.sample_h(vk)
    rbm.train(v0, vk, ph0, phk)
    train_loss += torch.mean(torch.abs(v0[v0>=0]-vk[v0>=0]))
    s += 1.
  print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.3469)
epoch: 2 loss: tensor(0.2432)
epoch: 3 loss: tensor(0.2454)
epoch: 4 loss: tensor(0.2505)
epoch: 5 loss: tensor(0.2496)
epoch: 6 loss: tensor(0.2484)
epoch: 7 loss: tensor(0.2485)
epoch: 8 loss: tensor(0.2469)
epoch: 9 loss: tensor(0.2494)
epoch: 10 loss: tensor(0.2478)


In [15]:
#Test RBM
test_loss = 0
s = 0.
for id_user in range(nb_users):
  v = training_set[id_user:id_user+1]
  vt = test_set[id_user:id_user+1]
  if len(vt[vt>=0]) > 0:
    _,h = rbm.sample_h(v)
    _,v = rbm.sample_v(h)
    test_loss += torch.mean(torch.abs(vt[vt>=0]-v[vt>=0]))
    s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2314)
